# Notebook 01: GenAI Extraction and Document Chunking

This notebook starts the first stage of the end to end GenAI insurance document pipeline. The goal is to take a collection of long, unstructured documents and convert them into structured JSON outputs that can be analysed in later notebooks.

The raw dataset includes three document types stored in the data/raw folder:  
• Insurance policy wording style documents  
• ESG style sustainability reports  
• Incident and loss investigation summaries  

Each document is synthetic, created through paraphrasing or original generation so that it mimics the style, tone, and complexity of real insurance documents without raising copyright issues. The documents are intentionally messy, unstructured, and varied in length to reflect the challenges seen in real underwriting and claims workflows.

Insurance related documents are often long and difficult to process. Important details such as entities, regions, risk factors, exclusions, operational failures, and ESG concerns are spread across multiple paragraphs. Manual extraction is slow and inconsistent. This notebook focuses on building a reproducible GenAI workflow that can extract structured fields in a reliable way.

In this notebook we will:
1. Load the raw documents from data/raw  
2. Split each document into smaller chunks for safer extraction  
3. Define a consistent JSON schema for the key fields we want to extract  
4. Build a clear prompt template that enforces structure and avoids hallucination  
5. Run the extraction process using helper functions in src  
6. Apply a simple MapReduce style pattern by merging chunk outputs  
7. Validate and save the final JSON outputs into data/processed  

The goal is to produce clean, validated, and consistent structured data. These outputs will then be used in Notebook 02 for EDA and in Notebook 03 for feature engineering and classification. This mirrors practical workflows in insurtech companies, where large volumes of unstructured documents must be prepared and structured before modelling or decision support.
